Audible Selenium Scraping Tutorial

https://www.youtube.com/watch?v=Zb-CrK04PmM&t=71s

## Google ColabでSeleniumスクレイピング

2023年の1月にはGoogle　ColabでSelenium使えてた、なのに1月後半にいろいろとあり以前と同じコードではColabでSeleniumが使えなくなった

原因はGoogleがUbuntuをアップデートしてSnapが無くなって
Snapが無いとChromium Webdriverが使えないのが問題らしく
いろいろ調べても多くのユーザーが困ってたのがこのビデオのおかげで解決した↓
https://www.youtube.com/watch?v=VSO14hgo6Gs

あとこのサイトがエラーの原因、対処法を分かりやすく書いてくれている https://kimudiary.com/python/google%E3%82%A2%E3%82%AB%E3%82%A6%E3%83%B3%E3%83%88%E3%81%A7%E4%BD%9C%E6%A5%AD%E3%81%AE%E8%87%AA%E5%8B%95%E5%8C%96%E3%80%90google-colaboratory%E3%81%A7web%E3%82%B9%E3%82%AF%E3%83%AC%E3%82%A4%E3%83%94/

## Install

In [ ]:
%%shell

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

Executing: /tmp/apt-key-gpghome.fyMuxlho1w/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.dsUVi0wv3N/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.YsXL5BbWDY/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1


In [ ]:
!apt-get update
!apt-get install chromium chromium-driver

Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian buster-updates InRelease [56.6 kB]
Get:3 http://deb.debian.org/debian-security buster/updates InRelease [34.8 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease
Get:7 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:8 http://deb.debian.org/debian buster/main amd64 Packages [10.7 MB]
Get:9 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:12 http://deb.debian.org/debian buster-updates/main amd64 Packages [9,745 B]
Get:13 http://deb.debian.org/debian-security buster/updates/main amd64 Packages [600 kB]

## Selenium

In [ ]:
!pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

import time
from time import sleep

import pandas as pd

In [ ]:
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox') # needed, because colab runs as root
    options.add_argument('--headless')  # or use pyvirtualdiplay
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

## Scrape 1 Scrape Audible

In [ ]:
driver = web_driver()

url = 'https://www.audible.com/adblbestsellers'

url2= 'https://www.audible.com/adblbestsellers?creativeId=23c3d2df-f714-4227-af60-c7bf8a7560d1&ipRedirectOverride=true&overrideBaseCountry=true&pageLoadId=f6pLAMSSrAREDOIv&page=2&ref=a_adblbests_c5_pageNext&pf_rd_p=e1595489-c152-4314-a5d7-ed60b7e2ecc8&pf_rd_r=PBA3B8N1NGSC2TED0N57&pageLoadId=fSicG7sf84v88E5R&creativeId=0bf0e03f-bb55-481b-b4fd-d67375977170'

driver.get(url)

# driver.quit()

In [ ]:
# driver.save_screenshot('1.jpg')

Now need to take care of the popup that appears and ask for signin with a Google Account

In [ ]:
# close_popup = driver.find_elements(By.ID, 'close')

# try:
#   close_popup.click()
# except:
#   pass

## exactly following youtube code

In [ ]:
# box = driver.find_elements(By.CSS_SELECTOR, '.bc-col-responsive .bc-col-6')


# get last page number

last_page = driver.find_elements(By.CSS_SELECTOR, '.bc-link.refinementFormLink.pageNumberElement.bc-color-link')
last_page_no = int(last_page[-1].text)
last_page_no

5

In [ ]:
# ratings

box = driver.find_elements(By.CSS_SELECTOR, '.bc-col-responsive .bc-col-6')

for i in box:
  try:
    rating = i.find_element(By.CSS_SELECTOR, '.bc-list-item .ratingsLabel span.bc-text.bc-pub-offscreen').text
    # print(ratings)
    print(rating.removesuffix(' out of 5 stars'))
    # print(type(language))
  except NoSuchElementException:
    print('Not rated yet')

#  div.bc-col-responsive.bc-spacing-top-none.bc-col-8 > div > div.bc-col-responsive.bc-col-6 > div > div > span > ul > li.bc-list-item.ratingsLabel > span.bc-text.bc-pub-offscreen

5
5
4.5
4.5
4.5
4.5
4.5
4.5
5
5
4.5
4.5
5
5
5
4.5
4.5
5
4.5
4.5


In [ ]:
# driver.quit()
# print'Error'

In [ ]:
page = ''
if page:
  print(page)
else:
  print('NONOONONOO')

NONOONONOO


Rating は新しい本だったりするとまだRatingが無いのでループ処理が止まってしまう、
なのでRatingがまだ無い場合、つまりそこだけ要素がない場合には（NoSuchElement)
NoSuchElementなら’Not rated yet'ってなるようにした

このやり方はこのサイトを参考にした↓
https://gist.github.com/webgyo/baa72dca8cbee6080b41ea2d14aebaad

In [ ]:
title_list, author_list, narrator_list, length_list, release_date_list, language_list, rating_list = [], [], [], [], [], [], []

for page in range(last_page_no):

# # delete later
# for page in range(2):


  print('printing page', page+1)
  box = driver.find_elements(By.CSS_SELECTOR, '.bc-col-responsive .bc-col-6')
  sleep(3)
  
  for item in box:
      
      # why find_element and not find_elements here?
      # because only trying to locate ONE h3 a in each item (only 1 h3 a per item) so thats why no need for find_elements and .text works
      # driver.save_screenshot('4.jpg')


    
      name = item.find_element(By.CSS_SELECTOR, 'h3 a').text
      # print(name)

      # returns list if multiple authors listed, so use find_elements
      authors = item.find_elements(By.CSS_SELECTOR, '.bc-list-item.authorLabel a')
      
      # if more than 1 auhor, returns like this - ['Jodi Picoult', 'Jennifer Finney Boylan']
      author = ' '.join([i.text for i in authors])
      # print(author)

      narrated_by = item.find_elements(By.CSS_SELECTOR, '.bc-list-item.narratorLabel a')
      narrator = ' '.join([i.text for i in narrated_by])


      length_audio = item.find_element(By.CSS_SELECTOR, '.bc-list-item.runtimeLabel span').text
      length = length_audio.removeprefix('Length: ')
      


      # working, changed this code as only 1 element per box so no need to use find_elements
      # length_audio = item.find_elements(By.CSS_SELECTOR, '.bc-list-item.runtimeLabel span')
      # length = [(i.text).removeprefix('Length: ') for i in length_audio]
      # print(type(length))
      # print(length)



      dates = item.find_element(By.CSS_SELECTOR, '.bc-list-item.releaseDateLabel span').text
      release_date = dates.removeprefix('Release date: ')

      language = item.find_element(By.CSS_SELECTOR, '.bc-list-item .languageLabel span').text
      language = language.removeprefix('Language: ')

      # some books, especially the new ones may not have ratings yet

      try:
          rating = item.find_element(By.CSS_SELECTOR, '.bc-list-item .ratingsLabel span.bc-text.bc-pub-offscreen').text
          # print(ratings)
          rating = rating.removesuffix(' out of 5 stars')
          print(rating)
      except NoSuchElementException:
          rating = 'Not rated yet'
          print('Not rated yet')






      # rating = item.find_element(By.CSS_SELECTOR, '.bc-list-item .ratingsLabel span.bc-text.bc-pub-offscreen').text
      # if rating:

      #   # rating = item.find_element(By.CSS_SELECTOR, '.bc-list-item .ratingsLabel span.bc-text.bc-pub-offscreen').text
      #   rating = rating.removesuffix(' out of 5 stars')
      #   print(rating)

      # else:
      #   rating = 'Not rated yet'
      #   print('Not rated yet')


      title_list.append(name)
      author_list.append(author)
      narrator_list.append(narrator)
      length_list.append(length)
      release_date_list.append(release_date)
      language_list.append(language)
      rating_list.append(rating)

  try:
    next = driver.find_element(By.CSS_SELECTOR, '.bc-button.bc-button-secondary.nextButton.refinementFormButton.bc-button-small.bc-button-inline')
    next.click()
    print('moving to next page')
    sleep(6)
  except:
    print('NO MORE NEXT PAGES')
    break
    

print("All Done")
driver.quit()
print(len(title_list))
print(len(author_list))

printing page 1
5
5
4.5
4.5
4.5
4.5
4.5
4.5
5
5
4.5
4.5
5
5
5
4.5
4.5
5
4.5
4.5
moving to next page
printing page 2
4.5
4.5
Not rated yet
4.5
5
4.5
5
5
5
4.5
5
5
4.5
5
5
4.5
5
5
5
4.5
moving to next page
printing page 3
5
5
5
5
4.5
4.5
5
5
4.5
4.5
5
4.5
4.5
5
5
5
5
5
5
4.5
moving to next page
printing page 4
4.5
5
4.5
5
5
4.5
5
5
4
4.5
4.5
5
5
4.5
4.5
4.5
5
5
5
4.5
moving to next page
printing page 5
4.5
4.5
4.5
5
4.5
4.5
5
4.5
5
4.5
4.5
5
5
5
5
4.5
4.5
5
5
5
NO MORE NEXT PAGES
All Done
100
100


In [ ]:
data = list(zip(title_list, author_list, narrator_list, length_list, release_date_list, language_list, rating_list))
data[:2]

[('Atomic Habits',
  'James Clear',
  'James Clear',
  '5 hrs and 35 mins',
  '10-16-18',
  'English',
  '5'),
 ('Spare',
  'Prince Harry The Duke of Sussex',
  'Prince Harry The Duke of Sussex',
  '15 hrs and 39 mins',
  '01-10-23',
  'English',
  '5')]

In [ ]:
import numpy as np

In [ ]:
file = pd.DataFrame(data, columns=['title', 'author', 'narrator', 'length', 'release_date_list', 'language', 'rating'])

# Rearrange index so that index works as the ranking of the books
file.index = np.arange(1, len(file) + 1)
file

,title,author,narrator,length,release_date_list,language,rating
1,Atomic Habits,James Clear,James Clear,5 hrs and 35 mins,10-16-18,English,5
2,Spare,Prince Harry The Duke of Sussex,Prince Harry The Duke of Sussex,15 hrs and 39 mins,01-10-23,English,5
3,Thank You for Listening,Julia Whelan,Julia Whelan,11 hrs and 16 mins,08-02-22,English,4.5
4,Lessons in Chemistry,Bonnie Garmus,Miranda Raison Bonnie Garmus Pandora Sykes,11 hrs and 55 mins,04-05-22,English,4.5
5,I Will Find You,Harlan Coben,Steven Weber,10 hrs and 16 mins,03-14-23,English,4.5
...,...,...,...,...,...,...,...
96,Reminders of Him,Colleen Hoover,Brittany Pressley Ryan West,10 hrs and 17 mins,01-18-22,English,4.5
97,Horse,Geraldine Brooks,James Fouhey Lisa Flanagan Graham Halstead Kat...,14 hrs and 6 mins,06-14-22,English,4.5
98,The Hobbit,J. R. R. Tolkien,Andy Serkis,10 hrs and 25 mins,09-21-20,English,5
99,The Midwife of Auschwitz,Anna Stuart,Sophie Aldred,11 hrs and 42 mins,05-31-22,English,5


In [ ]:
file[20:40]    # page2

,title,author,narrator,length,release_date_list,language,rating
21,It Starts with Us,Colleen Hoover,Colin Donnell Olivia Song,8 hrs and 41 mins,10-18-22,English,4.5
22,Young Forever,Dr. Mark Hyman MD,Dr. Mark Hyman MD,9 hrs and 28 mins,02-21-23,English,4.5
23,Outlive,Peter Attia MD Bill Gifford - contributor,Peter Attia MD,17 hrs and 7 mins,03-28-23,English,Not rated yet
24,A Court of Thorns and Roses,Sarah J. Maas,Jennifer Ikeda,16 hrs and 7 mins,05-05-15,English,4.5
25,Greenlights,Matthew McConaughey,Matthew McConaughey,6 hrs and 42 mins,10-20-20,English,5
26,Fairy Tale,Stephen King,Seth Numrich Stephen King,24 hrs and 6 mins,09-06-22,English,4.5
27,The Body Keeps the Score,Bessel A. van der Kolk,Sean Pratt,16 hrs and 15 mins,02-16-21,English,5
28,Paris,Paris Hilton,Paris Hilton,8 hrs and 22 mins,03-14-23,English,5
29,"Harry Potter and the Chamber of Secrets, Book 2",J.K. Rowling,Jim Dale,9 hrs and 2 mins,11-20-15,English,5
30,How to Win Friends & Influence People,Dale Carnegie,Andrew MacMillan,7 hrs and 15 mins,09-16-04,English,4.5


In [ ]:
file[40:60]    # should be page3 but printing page2 again

,title,author,narrator,length,release_date_list,language,rating
41,"Harry Potter and the Deathly Hallows, Book 7",J.K. Rowling,Jim Dale,21 hrs and 36 mins,11-20-15,English,5
42,Never Split the Difference,Chris Voss,Michael Kramer,8 hrs and 7 mins,05-17-16,English,5
43,Burner,Mark Greaney,Jay Snyder,16 hrs and 37 mins,02-21-23,English,5
44,A Court of Mist and Fury,Sarah J. Maas,Jennifer Ikeda,23 hrs and 16 mins,05-03-16,English,5
45,The Last Thing He Told Me,Laura Dave,Rebecca Lowman,8 hrs and 49 mins,05-04-21,English,4.5
46,Unfu*k Yourself,Gary John Bishop,Gary John Bishop,3 hrs and 24 mins,08-01-17,English,4.5
47,Finding Me,Viola Davis,Viola Davis,9 hrs and 15 mins,04-26-22,English,5
48,Project Hail Mary,Andy Weir,Ray Porter,16 hrs and 10 mins,05-04-21,English,5
49,Trial and Error,Robert Whitlow,Matthew Godfrey,11 hrs and 10 mins,02-02-21,English,4.5
50,Things We Hide from the Light,Lucy Score,Lila Winters Sebastian York,17 hrs and 36 mins,02-21-23,English,4.5


In [ ]:
file[60:80]    # page5

,title,author,narrator,length,release_date_list,language,rating
61,Things We Never Got Over,Lucy Score,Lila Winters Sebastian York,16 hrs and 1 min,03-01-22,English,4.5
62,Rich Dad Poor Dad: 20th Anniversary Edition,Robert T. Kiyosaki,Tom Parks,8 hrs and 22 mins,05-14-19,English,5
63,The Ballad of Songbirds and Snakes,Suzanne Collins,Santino Fontana,16 hrs and 16 mins,05-19-20,English,4.5
64,The Chronicles of Narnia Complete Audio Collec...,C. S. Lewis,Kenneth Branagh Alex Jennings Michael York Lyn...,33 hrs and 45 mins,12-27-19,English,5
65,Where the Crawdads Sing,Delia Owens,Cassandra Campbell,12 hrs and 12 mins,08-14-18,English,5
66,A Court of Silver Flames,Sarah J. Maas,Stina Nielsen,26 hrs and 5 mins,02-16-21,English,4.5
67,Magical Midlife Alliance,K.F. Breene,Nicole Poole,11 hrs and 26 mins,03-20-23,English,5
68,The Nightingale,Kristin Hannah,Polly Stone,17 hrs and 19 mins,02-03-15,English,5
69,I Have Some Questions for You,Rebecca Makkai,Julia Whelan JD Jackson,14 hrs and 4 mins,02-21-23,English,4
70,Think and Grow Rich,Napoleon Hill,Erik Synnestvedt,9 hrs and 35 mins,10-16-07,English,4.5


In [ ]:
# file.to_csv('audible_books', index=False)

In [ ]:
# driver.quit()

In [ ]:
######## ALL WORKING!!!!!!